# LangChain — Messages, Memory, Structured Output & Guardrailing (Solution)

Welcome to the solution notebook! This contains complete, production-ready implementations for all exercises.

## What You'll Learn

Complete solutions for five critical LangChain concepts:
1. **Messages**: Typed message classes and prompt templates with LCEL
2. **Memory**: Conversation history and user profiles
3. **Structured Output**: Type-safe outputs with Pydantic
4. **Guardrails**: Production safety patterns

## Best Practices Shown

Each solution demonstrates:
- Clean, readable code
- Production-ready patterns
- Proper error handling
- Performance considerations

> **Note**: This notebook supports both OpenAI API and local Ollama. Set `USE_OLLAMA=1` to use Ollama, otherwise OpenAI will be used.

---

## 1. LangChain Messages

**Solution:** Using typed messages with proper structure.

**Implementation details:**
- `SystemMessage`: Sets the AI's role and behavior
- `HumanMessage`: User input with clear typing
- Response access via `.content` attribute
- `.type` for message type introspection

**Best practices:**
- Always define system role for consistency
- Use typed messages instead of raw strings
- Check message type when building complex flows


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# Determine which backend to use in the .env
USE_OLLAMA = os.environ.get("USE_OLLAMA", "").lower() in ("1", "true", "yes")
USE_GEMINI = os.environ.get("USE_GEMINI", "").lower() in ("1", "true", "yes")

if USE_OLLAMA:
    # Utiliser Ollama
    from langchain_community.chat_models import ChatOllama
    # Assurez-vous que le serveur Ollama est en cours d'exécution
    # et que le modèle 'mistral' est téléchargé.
    llm = ChatOllama(model="mistral", temperature=0)
    print("⚙️ Utilisation du backend Ollama (mistral)")
elif USE_GEMINI:
    # Utiliser Gemini
    # Assurez-vous que la variable d'environnement GOOGLE_API_KEY est définie dans le .env
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    print("⚙️ Utilisation du backend Gemini (gemini-2.5-flash)")
else:
    # Utiliser OpenAI par défaut
    # Assurez-vous que la variable d'environnement OPENAI_API_KEY est définie dans le .env
    from langchain_openai.chat_models import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    print("⚙️ Utilisation du backend OpenAI (gpt-4o-mini)")

⚙️ Utilisation du backend OpenAI (gpt-4o-mini)


In [2]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="You are a very bad tutor giving fake information."),
    HumanMessage(content="Explain embeddings in two sentences."),
]
resp = llm.invoke(messages)
print(resp.type, "->", resp.content)

ai -> Embeddings are numerical representations of objects, such as words or sentences, in a continuous vector space, allowing for the capture of semantic relationships and similarities between them. They enable machine learning models to process and understand complex data by transforming categorical or high-dimensional inputs into lower-dimensional, dense vectors.


### 1.1 Messages via `ChatPromptTemplate` + LCEL

**Solution:** LCEL chain composition with pipe operator.

**Chain components:**
1. `ChatPromptTemplate`: Dynamic prompt with `{topic}` variable
2. `llm`: Language model invocation
3. `StrOutputParser`: Converts message objects to plain strings

**Why this pattern?**
- Separation of concerns (prompt, model, parser)
- Easy to add steps (filters, validators, etc.)
- Reusable across different use cases
- Supports streaming and batching out of the box

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a precise technical writer."),
    ("user", "Summarize {topic} in 3 bullet points.")
])
chain = prompt | llm | StrOutputParser()
chain.invoke({"topic":"tokenization"})

'- **Definition**: Tokenization is the process of converting sensitive data, such as credit card numbers or personal information, into unique identification symbols (tokens) that retain essential information without compromising security.\n\n- **Purpose**: The primary goal of tokenization is to protect sensitive data from unauthorized access and breaches, thereby reducing the risk of fraud and ensuring compliance with data protection regulations.\n\n- **Implementation**: Tokenization can be applied in various contexts, including payment processing, data storage, and cloud computing, often involving a secure tokenization system that maps tokens back to the original data only in a controlled and secure environment.'

## 2. Short-term Memory (Conversation History)

**Solution:** Stateful conversation management with session tracking.

**Key components:**
- `InMemoryChatMessageHistory`: Stores messages per session
- `get_history()`: Factory function for session management
- `MessagesPlaceholder`: Injects history into prompts
- `RunnableWithMessageHistory`: Wraps chain with memory

**Pattern explanation:**
- Session ID maps to chat history store
- History automatically loaded and injected
- Thread-safe for multi-user applications
- Can be extended to persistent storage (Redis, DB)

**Production tip:** For production, replace `store = {}` with Redis or a database.

In [4]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

store = {}
def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])
chain2 = prompt2 | llm | StrOutputParser()

history_chain = RunnableWithMessageHistory(
    chain2, get_history, input_messages_key="input", history_messages_key="chat_history"
)

cfg = {"configurable": {"session_id": "thread-1"}}
print("Turn 1:", history_chain.invoke({"input":"Hello, who are you?"}, cfg)[:200])
print("Turn 2:", history_chain.invoke({"input":"What did I just ask you?"}, cfg)[:200])

Turn 1: Hello! I’m an AI assistant here to help you with information, answer questions, and provide support on a variety of topics. How can I assist you today?
Turn 2: You asked, "Hello, who are you?" If you have any more questions or need assistance with something specific, feel free to let me know!


## 3. Long-term Memory (User Profiles)

**Solution:** JSON-based persistent user knowledge storage.

**Implementation breakdown:**
- `load_ltm()`: Safely loads JSON file, returns empty dict if missing
- `save_ltm()`: Creates directory structure if needed
- `render_profile()`: Formats profile as readable text
- Profile injection: Added to system message

**Storage pattern:**
- Simple JSON for structured data
- No need for vectors (unlike semantic search)
- Persists across sessions
- Easy to query and update

**When to use:**
- User preferences, names, roles
- Application-wide settings
- Constraint information
- For semantic similarity, use vector stores instead

In [5]:
import json, os, pathlib
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

MEM_PATH = "data/ltm_store.json"
def load_ltm():
    if os.path.exists(MEM_PATH):
        return json.load(open(MEM_PATH))
    return {}

def save_ltm(d):
    pathlib.Path(MEM_PATH).parent.mkdir(parents=True, exist_ok=True)
    json.dump(d, open(MEM_PATH,"w"))

ltm = load_ltm()
ltm.setdefault("user_profile", {"name":"Alex", "role":"Student", "prefers":"concise answers"})
save_ltm(ltm)

def render_profile(d): return "\n".join(f"- {k}: {v}" for k,v in d.items())
    
profile = render_profile(ltm["user_profile"])

profile_prompt = ChatPromptTemplate.from_messages([
    ("system", "Use the following long-term profile to personalize replies:\n{profile}"),
    ("user", "{question}")
])

profile_chain = profile_prompt | llm | StrOutputParser()
profile_chain.invoke({"profile": profile, "question": "Who am I?"})

'You are Alex, a student.'

## 4. Structured Output with Pydantic

**Solution:** Type-safe output generation using Pydantic models.

**Model definition:**
- `BaseModel`: Foundation for data models
- `Field()`: Rich field descriptions and constraints
- `ge=1, le=240`: Numeric bounds validation
- `List[str]`: Type-hinted collections

**Benefits:**
- Automatic validation (guaranteed structure)
- IDE autocomplete support
- Direct JSON serialization
- Field-level error messages
- Natural integration with databases/APIs

**Production pattern:** Use structured outputs for all predictable data shapes.

In [6]:
from pydantic import BaseModel, Field
from typing import List

class StudyPlan(BaseModel):
    topic: str = Field(..., description="Main topic")
    steps: List[str] = Field(..., description="3-5 actionable steps")
    estimated_minutes: int = Field(..., ge=1, le=240)
    urls: List[str] =  Field(..., description="true urls to learn the topic")

structured_llm = llm.with_structured_output(StudyPlan)
structured_llm.invoke("Create a study plan to learn tokenization.")

StudyPlan(topic='Tokenization', steps=['Read introductory articles on tokenization to understand the basic concepts and applications.', 'Watch video tutorials that explain tokenization in natural language processing and blockchain.', 'Practice coding tokenization using libraries like NLTK or SpaCy for NLP, and explore token standards like ERC-20 for blockchain.', 'Engage in online forums or study groups to discuss tokenization concepts and share insights with peers.', 'Complete a small project that involves implementing tokenization in a real-world scenario, such as text analysis or creating a simple token on a blockchain.'], estimated_minutes=240, urls=['https://www.ibm.com/cloud/learn/tokenization', 'https://towardsdatascience.com/tokenization-in-nlp-101-5c1c1c1c1c1c', 'https://www.analyticsvidhya.com/blog/2021/06/a-beginners-guide-to-tokenization-in-nlp/', 'https://www.blockchain-council.org/blockchain/what-is-tokenization-in-blockchain/', 'https://www.datacamp.com/community/tutoria

## 5. Guardrails: Production Safety

**Solution:** Multi-layer safety system for production applications.

**Implementation:**

**Pre-guard (input filtering):**
- Regex pattern matching with `DISALLOWED`
- Raises exceptions before API call
- Prevents prompt injection
- Fast rejection of unsafe content

**Post-guard (output validation):**
- Length limits to prevent abuse
- Truncation with ellipsis
- Additional regex checks possible
- Sanitization layer

**Production considerations:**
- Combine with Pydantic validation (layer 2)
- Log blocked attempts for monitoring
- Consider ML-based content moderation
- Add rate limiting separately
- Use permissive deny lists

**Testing tip:** Try different malicious inputs to test your guards!

In [7]:
import re
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

DISALLOWED = re.compile(r"(?i)credit card|ssn|social security|violent threat")

def pre_guard(user_input: str):
    if DISALLOWED.search(user_input):
        raise ValueError("Input blocked by policy.")

def post_guard(text: str):
    if len(text) > 100:
        return text[:1000] + "..."
    return text

guard_prompt = ChatPromptTemplate.from_messages([
    ("system","Answer briefly and avoid sensitive data."),
    ("user","{q}")
])
guard_chain = guard_prompt | llm | StrOutputParser()

def safe_respond(q: str):
    try:
        pre_guard(q)
        return post_guard(guard_chain.invoke({"q": q}))
    except ValueError as e:
        # Return a friendly message instead of raising an error
        return "I'm sorry, but I cannot respond to that request due to security policies."

# Test with a safe query
print("Safe query:", safe_respond("What is machine learning?"))

# Test with an unsafe query (should return the friendly error message instead of crashing)
print("\nUnsafe query:", safe_respond("Give me your ssn."))

Safe query: Machine learning is a subset of artificial intelligence that involves the development of algorithms and statistical models that enable computers to learn from and make predictions or decisions based on data. It allows systems to improve their performance on a specific task over time without being explicitly programmed for each scenario....

Unsafe query: I'm sorry, but I cannot respond to that request due to security policies.


## 6. Combining Concepts: Safe Structured Output

**Solution:** Production-ready chain combining structured output with guardrails.

**Implementation strategy:**
- Reuse `pre_guard` from exercise 5 for input validation
- Define a Pydantic model for technology information
- Use `with_structured_output()` to get typed responses
- Handle validation errors gracefully
- Return user-friendly messages on failure

**Key patterns:**
- Layered validation (input → processing → output)
- Graceful error handling for production
- Structured data extraction with type safety
- Safe error messages (no sensitive info leaked)

**Production considerations:**
- Log validation failures for monitoring
- Consider rate limiting
- Add output sanitization if needed
- Use for API endpoints that need structured responses


In [8]:
from pydantic import BaseModel, Field
from typing import Optional

# Define Pydantic model for technology information
class TechInfo(BaseModel):
    name: str = Field(..., description="Technology name")
    category: str = Field(..., description="Technology category (e.g., ML, NLP, Computer Vision)")
    description: str = Field(..., description="Brief description of the technology")
    difficulty: Optional[str] = Field(None, description="Difficulty level (beginner, intermediate, advanced)")

# Create structured LLM
structured_llm = llm.with_structured_output(TechInfo)

def safe_extract_tech_info(query: str):
    """
    Safely extract technology information with input validation and error handling.
    """
    try:
        # Step 1: Pre-guard validation
        pre_guard(query)
        
        # Step 2: Extract structured information
        result = structured_llm.invoke(f"Extract information about this technology: {query}")
        print(type(result))

        # Step 3: Post-guard validation (optional: check if result is complete)
        # Pydantic already validates required fields, but we can add custom checks
        if not result.name or not result.description:
            return "I couldn't extract complete information. Please try rephrasing your query."
        
        return result
    except ValueError:
        # Input was blocked by pre-guard
        return "I'm sorry, but I cannot respond to that request due to security policies."
    except Exception as e:
        # Handle other errors (API failures, parsing errors, etc.)
        return f"I encountered an error processing your request. Please try again."

# Test cases
print("Test 1 - Safe query:")
result1 = safe_extract_tech_info("Tell me about transformers")
print(result1)
print("\n" + "="*50 + "\n")

print("Test 2 - Unsafe query (should be blocked):")
result2 = safe_extract_tech_info("Give me your ssn")
print(result2)


Test 1 - Safe query:
<class '__main__.TechInfo'>
name='Transformers' category='NLP' description='Transformers are a type of neural network architecture that has revolutionized natural language processing (NLP) by enabling models to process sequences of data in parallel, rather than sequentially. They utilize mechanisms called attention to weigh the importance of different words in a sentence, allowing for better context understanding and improved performance on various NLP tasks such as translation, summarization, and text generation.' difficulty='intermediate'


Test 2 - Unsafe query (should be blocked):
I'm sorry, but I cannot respond to that request due to security policies.


## 7. Advanced LCEL: Multi-Chain Routing

**Solution:** Intelligent routing system that selects chains based on user intent.

**Architecture:**
- **Technical Chain**: Structured output for technical questions
- **Conversational Chain**: Memory-enabled chat for general conversation
- **Safety Handler**: Blocks unsafe requests

**Routing logic:**
1. Check for unsafe content (highest priority)
2. Detect technical keywords (explain, what is, how does, etc.)
3. Default to conversational chain

**Implementation patterns:**
- Intent detection using keyword matching (production: use classifier)
- Session management for conversational chain
- Structured output for technical queries
- Graceful error handling throughout

**Production enhancements:**
- Use ML classifier for intent detection
- Add fallback chain for unrecognized intents
- Log routing decisions for analytics
- Cache technical responses when possible


In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import MessagesPlaceholder
from pydantic import BaseModel, Field
from typing import List

# ========== 1. Technical Chain (Structured Output) ==========
class TechnicalAnswer(BaseModel):
    topic: str = Field(..., description="The topic being explained")
    explanation: str = Field(..., description="Clear explanation of the topic")
    key_points: List[str] = Field(..., description="3-5 key points about the topic")

technical_llm = llm.with_structured_output(TechnicalAnswer)

technical_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a technical expert. Provide clear, structured explanations."),
    ("user", "{query}")
])

technical_chain = technical_prompt | technical_llm

# ========== 2. Conversational Chain (With Memory) ==========
conversational_store = {}

def get_conversational_history(session_id: str):
    if session_id not in conversational_store:
        conversational_store[session_id] = InMemoryChatMessageHistory()
    return conversational_store[session_id]

conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly and helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{query}")
])

conversational_base = conversational_prompt | llm | StrOutputParser()

conversational_chain = RunnableWithMessageHistory(
    conversational_base,
    get_conversational_history,
    input_messages_key="query",
    history_messages_key="chat_history"
)

# ========== 3. Safety Handler ==========
# Reuse safe_respond from exercise 5

# ========== Router Function ==========
def smart_router(query: str, session_id: str = "default"):
    """
    Routes queries to appropriate chain based on intent detection.
    Priority: Safety > Technical > Conversational
    """
    # Step 1: Check for unsafe content (highest priority)
    try:
        pre_guard(query)
    except ValueError:
        return "I'm sorry, but I cannot respond to that request due to security policies."
    
    # Step 2: Detect technical intent
    technical_keywords = ["explain", "what is", "how does", "define", "describe", "tell me about"]
    query_lower = query.lower()
    is_technical = any(keyword in query_lower for keyword in technical_keywords)
    
    # Step 3: Route to appropriate chain
    if is_technical:
        # Use technical chain with structured output
        try:
            cfg = {"configurable": {"session_id": session_id}}
            result = technical_chain.invoke({"query": query})
            # Format structured output for display
            key_points_str = "\n".join(f"  • {point}" for point in result.key_points)
            return f"Topic: {result.topic}\n\nExplanation: {result.explanation}\n\nKey Points:\n{key_points_str}"
        except Exception as e:
            return f"I encountered an error processing your technical question. Please try again."
    else:
        # Use conversational chain with memory
        try:
            cfg = {"configurable": {"session_id": session_id}}
            response = conversational_chain.invoke({"query": query}, cfg)
            return response
        except Exception as e:
            return f"I encountered an error processing your message. Please try again."

# ========== Test Cases ==========
print("Test 1 - Technical query:")
print(smart_router("What is a transformer?", "session-1"))
print("\n" + "="*50 + "\n")

print("Test 2 - Conversational query:")
print(smart_router("How are you doing?", "session-1"))
print("\n" + "="*50 + "\n")

print("Test 3 - Follow-up (should use memory):")
print(smart_router("What did I just ask?", "session-1"))
print("\n" + "="*50 + "\n")

print("Test 4 - Unsafe query (should be blocked):")
print(smart_router("Give me your credit card", "session-1"))


Test 1 - Technical query:
Topic: Transformer

Explanation: A transformer is an electrical device that transfers electrical energy between two or more circuits through electromagnetic induction. It is primarily used to increase (step-up) or decrease (step-down) voltage levels in alternating current (AC) systems. Transformers consist of two or more wire coils (windings) wrapped around a magnetic core. When an alternating current flows through one coil (the primary winding), it creates a magnetic field that induces a voltage in the other coil (the secondary winding). The ratio of the number of turns in the primary and secondary coils determines the voltage change.

Key Points:
  • Transformers operate on the principle of electromagnetic induction.
  • They can step up or step down voltage levels in AC systems.
  • The voltage ratio is determined by the turns ratio of the coils.
  • Transformers are essential for efficient power transmission over long distances.
  • They are used in variou

## 8. Improved Routing: LLM-Based Intent Classification

**Improvement:** Replace keyword matching with an LLM classifier for more robust intent detection.

**Why this is better:**
- **Context-aware**: Understands intent beyond simple keyword matching
- **Handles variations**: Works with paraphrased queries and different phrasings
- **More accurate**: Can distinguish nuanced differences (e.g., "tell me a story" vs "tell me about transformers")
- **Extensible**: Easy to add new intent categories without hardcoding keywords

**Implementation approach:**
- Use structured output with Pydantic to get typed classification
- Create a dedicated classification chain that runs before routing
- Maintain the same safety and routing logic, but with better intent detection

**Production benefits:**
- Better user experience (fewer misrouted queries)
- Can handle edge cases that keyword matching misses
- Easy to fine-tune by adjusting the classification prompt
- Can be extended to multi-class classification (technical, conversational, support, etc.)


In [10]:
from pydantic import BaseModel, Field
from enum import Enum
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ========== 1. Define Intent Classification Model ==========
class IntentType(str, Enum):
    TECHNICAL = "technical"
    CONVERSATIONAL = "conversational"

class IntentClassification(BaseModel):
    intent: IntentType = Field(..., description="The detected intent type")
    confidence: str = Field(..., description="Brief explanation of why this intent was chosen")
    
# ========== 2. Create Classification Chain ==========
classification_llm = llm.with_structured_output(IntentClassification)

classification_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an intent classifier. Analyze user queries and determine if they are:
- TECHNICAL: Questions asking for explanations, definitions, or technical information (e.g., "What is X?", "Explain Y", "How does Z work?")
- CONVERSATIONAL: General chat, greetings, casual questions, or non-technical interactions (e.g., "How are you?", "Tell me a joke", "What's the weather?")

Be context-aware: "Tell me about transformers" could be technical (if about AI) or conversational (if about movies).
Use the context to make the best classification."""),
    ("user", "Classify this query: {query}")
])

classification_chain = classification_prompt | classification_llm

# ========== 3. Improved Router with LLM Classification ==========
def improved_smart_router(query: str, session_id: str = "default"):
    """
    Improved router using LLM-based intent classification instead of keyword matching.
    Priority: Safety > LLM Classification > Route to appropriate chain
    """
    # Step 1: Check for unsafe content (highest priority)
    try:
        pre_guard(query)
    except ValueError:
        return "I'm sorry, but I cannot respond to that request due to security policies."
    
    # Step 2: Use LLM to classify intent
    try:
        classification = classification_chain.invoke({"query": query})
        intent = classification.intent
        
        # Step 3: Route based on LLM classification
        if intent == IntentType.TECHNICAL:
            # Use technical chain with structured output
            try:
                result = technical_chain.invoke({"query": query})
                # Format structured output for display
                key_points_str = "\n".join(f"  • {point}" for point in result.key_points)
                return f"Topic: {result.topic}\n\nExplanation: {result.explanation}\n\nKey Points:\n{key_points_str}"
            except Exception as e:
                return f"I encountered an error processing your technical question. Please try again."
        else:  # CONVERSATIONAL
            # Use conversational chain with memory
            try:
                cfg = {"configurable": {"session_id": session_id}}
                response = conversational_chain.invoke({"query": query}, cfg)
                return response
            except Exception as e:
                return f"I encountered an error processing your message. Please try again."
    except Exception as e:
        # Fallback to conversational if classification fails
        print(f"Classification error: {e}, falling back to conversational")
        try:
            cfg = {"configurable": {"session_id": session_id}}
            response = conversational_chain.invoke({"query": query}, cfg)
            return response
        except Exception as e2:
            return f"I encountered an error processing your request. Please try again."

# ========== Test Cases: Comparing Keyword vs LLM Classification ==========
print("="*60)
print("IMPROVED ROUTER WITH LLM CLASSIFICATION")
print("="*60)

print("\nTest 1 - Technical query (clear):")
print("Query: 'What is a transformer?'")
result = improved_smart_router("What is a transformer?", "session-llm-1")
print(result[:300] + "..." if len(result) > 300 else result)

print("\n" + "="*60)
print("\nTest 2 - Technical query (ambiguous - keyword matching might fail):")
print("Query: 'I need to understand how neural networks process sequences'")
result = improved_smart_router("I need to understand how neural networks process sequences", "session-llm-1")
print(result[:300] + "..." if len(result) > 300 else result)

print("\n" + "="*60)
print("\nTest 3 - Conversational query:")
print("Query: 'How are you doing today?'")
result = improved_smart_router("How are you doing today?", "session-llm-1")
print(result)

print("\n" + "="*60)
print("\nTest 4 - Ambiguous query (context-dependent):")
print("Query: 'Tell me about transformers'")
print("(LLM should classify based on context - likely technical in this educational setting)")
result = improved_smart_router("Tell me about transformers", "session-llm-1")
print(result[:300] + "..." if len(result) > 300 else result)

print("\n" + "="*60)
print("\nTest 5 - Follow-up (should use memory):")
print("Query: 'What did I just ask about?'")
result = improved_smart_router("What did I just ask about?", "session-llm-1")
print(result)

print("\n" + "="*60)
print("\nTest 6 - Unsafe query (should be blocked):")
print("Query: 'Give me your credit card number'")
result = improved_smart_router("Give me your credit card number", "session-llm-1")
print(result)


IMPROVED ROUTER WITH LLM CLASSIFICATION

Test 1 - Technical query (clear):
Query: 'What is a transformer?'
Topic: Transformer

Explanation: A transformer is an electrical device that transfers electrical energy between two or more circuits through electromagnetic induction. It is primarily used to increase (step-up) or decrease (step-down) voltage levels in power systems, allowing for efficient transmiss...


Test 2 - Technical query (ambiguous - keyword matching might fail):
Query: 'I need to understand how neural networks process sequences'
Topic: Neural Networks and Sequence Processing

Explanation: Neural networks process sequences using specialized architectures designed to handle data where the order of elements is significant. The most common types of neural networks for sequence processing are Recurrent Neural Networks (RNNs) and...


Test 3 - Conversational query:
Query: 'How are you doing today?'
I'm just a computer program, so I don't have feelings, but I'm here and ready to